In [1]:
!pip install chromadb
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 40.

In [2]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModel
from torch import Tensor
import os
from chromadb.config import Settings
from chromadb.api.models.Collection import Collection
import pandas as pd
import chromadb
import numpy as np
from sentence_transformers import SentenceTransformer


In [ ]:
PATH_EMBEDDING = "/content/chroma" #Unzip embbeing
client = chromadb.PersistentClient(path=PATH_EMBEDDING)
collection = client.get_or_create_collection("test")

In [ ]:
sentence_model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [ ]:
print(collection.count())

24690


In [ ]:
def encodeQuestion(question, model, collection):
  embedding = model.encode(question, normalize_embeddings=True)
  embedding_list = embedding.tolist()
  result = collection.query(query_embeddings=[embedding_list])
  result_documents = []

  for distance, metadata, document in zip(
          result["distances"][0], result["metadatas"][0], result["documents"][0]
  ):
      if distance < 1:
          result_documents.append({"file_path": metadata["file_path"], "answer": document, "metric":distance})
  result_documents.sort(key=lambda x: x["metric"], reverse=False)
  result_documents = result_documents[:3]

  return result_documents

In [ ]:
qa_template = """
        <SC1>Вы - банковский ассистент. Вы отвечаете на вопрос с помощью информации,вы можете использовать ее при ответе если вопрос.
        Отвечай только на то, о чем тебя спросили. Не задавай встречных вопросов.
        Не повторяй информацию, только отвечай на вопрос. Давай один точный ответ только по информации. Не повторяй вопрос
        Информация: {context}
        <SC6> Вопрос: {question}
        <extra_id_0>
        """

In [ ]:
# Фунцкия нужна для правильного формирования ответа для пользователя
def remove_after_question(text):
    question_pos = text.find("Вопрос :")
    if question_pos != -1:
        return text[:question_pos]
    else:
        return text

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)


tokenizer_config.json:   0%|          | 0.00/20.2k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.96G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


T5ForConditionalGeneration(
  (shared): Embedding(50364, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
       

In [ ]:
while True:
  question = input("Введите запрос: ")
  top_answer = encodeQuestion(question, sentence_model,collection)
  result = []
  for answer in top_answer:
    result.append(answer['answer'])
  if len(top_answer) == 0:
    qa = qa_template.format(context="", question=question)
  else:
    qa = qa_template.format(context=result, question=question)
  input_ids=torch.tensor([tokenizer.encode(qa, add_special_tokens=True)]).to(device)
  outputs=model.generate(input_ids,
                            top_p = 0.4,
                            temperature=0.2,
                            repetition_penalty =2.0,
                            min_length = 50,
                            max_length = 800,
                            pad_token_id=tokenizer.eos_token_id,
                            do_sample=True)
  new_text = remove_after_question(tokenizer.decode(outputs[0][1:]))
  print(new_text)